In [2]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE
def get_market_membership_from_excel(path_msci, convert_to_daily = False):
    ### Importing standard modules and date-special modules:    
    import numpy as np
    import pandas as pd
    ### Reindexing function declaring:
    def reindex_month_ends(iter_group):
        iter_range = pd.date_range(iter_group.first_valid_index(), iter_group.last_valid_index(), freq = 'BM')
        iter_result = iter_group.reindex(iter_range)
        return iter_result    
    ### Declaring local constants & variables: 
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Reindexing to show absent monthes for future daily resampling: 
    if (convert_to_daily):
        ser_market_membership = ser_market_membership.groupby('Country').apply(lambda iter_group: reindex_month_ends(iter_group.droplevel(1)))
        ser_market_membership.index.names = ['Country', 'Date']
        ser_market_membership = ser_market_membership.swaplevel()
        ser_market_membership = ser_market_membership.reset_index('Country').groupby('Country').resample('B').ffill().drop('Country', axis = 1).squeeze()
        ser_market_membership = ser_market_membership.swaplevel().sort_index(level = ['Country', 'Date'])
        
    return ser_market_membership

In [3]:
def get_cds_from_excel(path_cds, path_msci):
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd 
    ### Declaring global constants & variables: 
    arr_cds_source = ['CBIN', 'CMAN']
    dict_cds = {}
    ### Collecting sheets:
    with pd.ExcelFile(path_cds) as file_excel:
        for str_sheet_name in file_excel.sheet_names:
            ### Sheet checking and reading:
            if (str_sheet_name.upper() in arr_cds_source):
                df_source = pd.read_excel(io = file_excel, sheet_name = str_sheet_name, skiprows = list(range(9)), header = 0, index_col = [0], parse_dates = True, 
                                          na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                                       '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null', '#N/A Invalid Security'], keep_default_na = False) 
                ### Table mungling:
                ser_cds = df_source.stack(dropna = False).squeeze()
                ser_cds.name = str_sheet_name
                dict_cds[str_sheet_name.upper()] = ser_cds.sort_index()
    ### Aggregating tables:                
    df_cds = pd.concat(dict_cds, axis = 1).sort_index()
    df_cds.index.names = ['Date', 'Country']
    ### Receiving MSCI membership data:    
    ser_market_membership = get_market_membership_from_excel(path_msci, convert_to_daily = True)
    ### Adding membership column to result dataframe:    
    df_cds = df_cds.join(ser_market_membership, on = ['Date', 'Country'], how = 'left')
    df_cds = df_cds.set_index('Market', drop = True, append = True) 
 
    return df_cds

In [4]:
### ALL RATINGS IMPORTING
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
### Declaring global constants & variables: 
path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path to membership source
path_cds = 'Data_Files/Source_Files/CDS_Spreadsheet.xlsx' ### Path to CDS data file
path_market_cap = 'Data_Files/Source_Files/Market_Cap.h5'
key_market_cap = 'mcap'
All = slice(None)
### Market caps extracting:
ser_mcap = pd.read_hdf(path_market_cap, key_market_cap)
ser_mcap_daily = ser_mcap.groupby('Country').apply(lambda iter_group: iter_group.droplevel([1, 2]).resample('B').bfill())
### Receiving aggregated ratings table:
df_cds = get_cds_from_excel(path_cds, path_msci)
### Joining market caps:
df_cds_mcap = df_cds.reset_index('Market').join(ser_mcap_daily.swaplevel()).set_index('Market', append = True)

In [7]:
### EXPORT RESULTS SAVING

### Declaring constants:
path_cds_hdf = 'Data_Files/Source_Files/CDS_Data.h5'
key_cds_mcap = 'cds_mcap'
### Saving to hdf:
df_cds_mcap.to_hdf(path_cds_hdf, key_cds_mcap, mode = 'w', format = 'fixed')

In [ ]:
### EXPLORING CDS DATA
import matplotlib.pyplot as plt
### Declaring global constants & variables: 
arr_cds_source = ['CBIN', 'CMAN']
arr_market = [All, ['DM', 'EM', 'FM'], ['DM'], ['EM'], ['FM']]
arr_market_str = ['ANY REGION', 'DM + EM + FM', 'DM only', 'EM only', 'FM only']
arr_period = [999, 24, 12]
arr_period_str = ['ALL DATES', 'LAST TWO YEARS', 'LAST YEAR']
### Testing values:
#arr_cds_source = ['CBIN']
#arr_market = [['DM']]
#arr_market_str = ['DM only']
#arr_period = [12]
#arr_period_str = ['LAST YEAR']
### Defining function for internal missing observations counting:
def get_missing_observations(iter_group, iter_last_date):
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd  
    ### Internal gaps calculation:
    iter_group = iter_group.droplevel([1, 2])
    if (len(iter_group.dropna()) > 0):
        iter_start_date = iter_group.first_valid_index()        
        iter_date_range = pd.date_range(start = iter_start_date, end = iter_last_date, freq = 'B')
        iter_result = iter_group.count() / len(iter_date_range)
    else:
        iter_result = np.NaN
    return iter_result
### Defining function for market cap weighting:
def set_mcap_weights(iter_group):
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd 
    ### Normalizing by mcap:
#   iter_group.loc[iter_group[iter_source].isna(), 'Market Cap'] = 0
    iter_result = iter_group[iter_source].mul(iter_group['Market Cap'].pow(0.5))
    iter_result = iter_result / (iter_group['Market Cap'].pow(0.5)).sum()  
    return iter_result
### Main loop
for iter_source in arr_cds_source:
    ### Preparing market cap weighted CDS quantity counter:
    df_cds_weighted = df_cds_mcap[[iter_source, 'Market Cap']].copy()
    df_cds_weighted.loc[df_cds_weighted[iter_source].notna(), iter_source] = 1
    ### Looping periods:
    for iter_period_num, iter_period in enumerate(arr_period):
        ### Looping markets:
        for iter_market_num, iter_market in enumerate(arr_market):
            ### Defining index:
            iter_prefix = iter_source + ' : ' + arr_period_str[iter_period_num] + ' : ' + arr_market_str[iter_market_num]
            ### Selecting appropriate data set:
            ser_iter_cds = df_cds_mcap[iter_source].copy()
            iter_last_date = ser_iter_cds.index.get_level_values(0).max()
            ser_iter_cds = ser_iter_cds.loc[iter_last_date - pd.offsets.BMonthEnd(iter_period) + pd.offsets.BDay(): iter_last_date, All, iter_market]
            ### Calculating marketcap weighted counter:
            df_iter_cds_weighted = df_cds_weighted.loc[ser_iter_cds.index, All]
            ser_iter_weighted = df_iter_cds_weighted.groupby('Date').apply(lambda iter_group: set_mcap_weights(iter_group)).droplevel(0)                    
            ### Calculating nominal counters:
            ser_iter_country_count = ser_iter_cds.groupby('Country').count()
            ser_iter_date_proportion = ser_iter_cds.groupby('Date').apply(lambda iter_group: iter_group.count() / len(iter_group.index))
            ser_iter_country_missed = ser_iter_cds.groupby('Country').apply(lambda iter_group: get_missing_observations(iter_group, iter_last_date))
            ser_iter_country_missed = ser_iter_country_missed.dropna()    
            ### Results output:
            print(iter_prefix, ': Data set date range:', ser_iter_cds.dropna().index.get_level_values(0).min().date(), '-',
                  ser_iter_cds.dropna().index.get_level_values(0).max().date(), '(', len(ser_iter_cds.dropna().index.get_level_values(0).unique()), 'dates )')
            print(iter_prefix, ': Data set coverage:', '{:.2%}'.format(ser_iter_cds.count() / len(ser_iter_cds.index)), '(', ser_iter_cds.count(), ')')
            print(iter_prefix, ': Average country\'s number per date:', round(ser_iter_cds.groupby('Date').count().mean(), 2))
            print(iter_prefix, ': Number of countries with at least one effective observation:', 
                  ser_iter_cds.groupby('Country').count()[ser_iter_cds.groupby('Country').count() > 0].count())        
            print(iter_prefix, ': Average number of valid observations for country:', int(ser_iter_country_count.mean()), 
                  '(', '{:.2%}'.format(ser_iter_country_count.mean() / len(ser_iter_cds.dropna().index.get_level_values(0).unique())), 'of dates number)')
            print(iter_prefix, ': Average number of valid observations for country with at least one valid observation:', 
                  int(ser_iter_country_count[ser_iter_country_count > 0].mean()), 
                  '(', '{:.2%}'.format(ser_iter_country_count[ser_iter_country_count > 0].mean() / len(ser_iter_cds.dropna().index.get_level_values(0).unique())), ')')  
            print(iter_prefix, ': Average proportion of coverage after first valid observation to the last available data set date:', 
                  '{:.2%}'.format(ser_iter_country_missed.mean()))
            print(iter_prefix, ': Average proportion of coverage per date:', '{:.2%}'.format(ser_iter_date_proportion.mean()), 'of countries number') 
            print(iter_prefix, ': Average proportion of coverage per date weighted by market cap square rooted:', 
                  '{:.2%}'.format(ser_iter_weighted.groupby('Date').sum().mean()), 'of weighted countries number')       
            
            ser_coverage_to_show = pd.concat([ser_iter_date_proportion.resample('BM').mean(), ser_iter_weighted.groupby('Date').sum().resample('BM').mean()], axis = 1)
            ser_coverage_to_show.columns = ['Simple coverage', 'Weighted coverage']
            ser_coverage_to_show.plot(figsize = (20, 5), title = iter_prefix + ': Coverage evolution (month average)')
            plt.show()             

In [ ]:
### CDS DTA SOURCES COMPARING
import matplotlib.pyplot as plt
### Declaring global constants & variables: 
arr_market = [All, ['DM', 'EM', 'FM'], ['DM'], ['EM'], ['FM']]
arr_market_str = ['ANY REGION', 'DM + EM + FM', 'DM only', 'EM only', 'FM only']
arr_period = [999, 24, 12]
arr_period_str = ['ALL DATES', 'LAST TWO YEARS', 'LAST YEAR']
### Testing values:
arr_market = [['DM']]
arr_market_str = ['DM']
arr_period = [999]
arr_period_str = ['ALL DATES']
### Looping periods:
for iter_period_num, iter_period in enumerate(arr_period):
    ### Looping markets:
    for iter_market_num, iter_market in enumerate(arr_market):
        ### Defining index:
        iter_prefix = arr_period_str[iter_period_num] + ' : ' + arr_market_str[iter_market_num]
        ### Selecting appropriate data set:
        iter_last_date = df_cds_mcap.index.get_level_values(0).max()
        iter_range = pd.date_range(iter_last_date - pd.offsets.BMonthEnd(iter_period) + pd.offsets.BDay(), iter_last_date, freq = 'B')
        df_iter_cds = df_cds_mcap.loc[(iter_range, All, iter_market), :]
        ### Reasults output:
        ser_iter_cbin = df_iter_cds.loc[df_iter_cds['CBIN'].notna()]
        print(iter_prefix, ': CBIN covered observations :', '{:.2%}'.format(len(ser_iter_cbin.index) / len(df_iter_cds.index)),
              '(', len(ser_iter_cbin.index), 'observations from', len(df_iter_cds.index), ')')  
        print(iter_prefix, ': CBIN covered :', 'Proportion of dates presence :', 
              '{:.2%}'.format(len(ser_iter_cbin.index.get_level_values(0).unique()) / len(df_iter_cds.index.get_level_values(0).unique())),
              '(', len(ser_iter_cbin.index.get_level_values(0).unique()), 'dates from', len(df_iter_cds.index.get_level_values(0).unique()), ')')
        print(iter_prefix, ': CBIN covered :', 'Proportion of countries presence :', 
              '{:.2%}'.format(len(ser_iter_cbin.index.get_level_values(1).unique()) / len(df_iter_cds.index.get_level_values(1).unique())),
              '(', len(ser_iter_cbin.index.get_level_values(1).unique()), 'countries from', len(df_iter_cds.index.get_level_values(1).unique()), ')')         
        ser_iter_cman = df_iter_cds.loc[df_iter_cds['CMAN'].notna()]        
        print(iter_prefix, ': CMAN covered observations :', '{:.2%}'.format(len(ser_iter_cman.index) / len(df_iter_cds.index)),
              '(', len(ser_iter_cman.index), 'observations from', len(df_iter_cds.index), ')')
        print(iter_prefix, ': CMAN covered :', 'Proportion of dates presence :', 
              '{:.2%}'.format(len(ser_iter_cman.index.get_level_values(0).unique()) / len(df_iter_cds.index.get_level_values(0).unique())),
              '(', len(ser_iter_cman.index.get_level_values(0).unique()), 'dates from', len(df_iter_cds.index.get_level_values(0).unique()), ')')
        print(iter_prefix, ': CMAN covered :', 'Proportion of countries presence :', 
              '{:.2%}'.format(len(ser_iter_cman.index.get_level_values(1).unique()) / len(df_iter_cds.index.get_level_values(1).unique())),
              '(', len(ser_iter_cman.index.get_level_values(1).unique()), 'countries from', len(df_iter_cds.index.get_level_values(1).unique()), ')')                 
        print(iter_prefix, ': Both CMAN absent and CBIN absent :', 
              '{:.2%}'.format(len(df_iter_cds.loc[df_iter_cds['CMAN'].isna() & df_iter_cds['CBIN'].isna()].index) / len(df_iter_cds.index)),
              '(', len(df_iter_cds.loc[df_iter_cds['CMAN'].isna() & df_iter_cds['CBIN'].isna()].index), 'observations from', len(df_iter_cds.index), ')')
        print(iter_prefix, ': CMAN :', 'Empty countries number :', 
              df_iter_cds['CMAN'].groupby('Country').apply(lambda iter_group: 1 if iter_group.count() == 0 else 0).sum())        
        print(iter_prefix, ': CMAN :', 'Empty dates number :', 
              df_iter_cds['CMAN'].groupby('Date').apply(lambda iter_group: 1 if iter_group.count() == 0 else 0).sum())  
        print(iter_prefix, ': CBIN :', 'Empty countries number :', 
              df_iter_cds['CBIN'].groupby('Country').apply(lambda iter_group: 1 if iter_group.count() == 0 else 0).sum())        
        print(iter_prefix, ': CBIN :', 'Empty dates number :', 
              df_iter_cds['CBIN'].groupby('Date').apply(lambda iter_group: 1 if iter_group.count() == 0 else 0).sum())  
        ser_iter_cbin_only = df_iter_cds.loc[df_iter_cds['CMAN'].isna() & df_iter_cds['CBIN'].notna()]['CBIN']        
        print(iter_prefix, ': CMAN absent and CBIN covered :', 
              '{:.2%}'.format(len(ser_iter_cbin_only.index) / len(df_iter_cds.index)),
              '(', len(ser_iter_cbin_only.index), 'observations from', len(df_iter_cds.index), ')')   
        print(iter_prefix, ': CBIN only covered :', 'Proportion of dates presence :', 
              '{:.2%}'.format(len(ser_iter_cbin_only.index.get_level_values(0).unique()) / len(df_iter_cds.index.get_level_values(0).unique())),
              '(', len(ser_iter_cbin_only.index.get_level_values(0).unique()), 'dates from', len(df_iter_cds.index.get_level_values(0).unique()), ')')
        print(iter_prefix, ': CBIN only covered :', 'Proportion of countries presence :', 
              '{:.2%}'.format(len(ser_iter_cbin_only.index.get_level_values(1).unique()) / len(df_iter_cds.index.get_level_values(1).unique())),
              '(', len(ser_iter_cbin_only.index.get_level_values(1).unique()), 'countries from', len(df_iter_cds.index.get_level_values(1).unique()), ')')          
        ser_iter_cman_only = df_iter_cds.loc[df_iter_cds['CMAN'].notna() & df_iter_cds['CBIN'].isna()]['CMAN']
        print(iter_prefix, ': CMAN covered and CBIN absent :', 
              '{:.2%}'.format(len(ser_iter_cman_only.index) / len(df_iter_cds.index)),
              '(', len(ser_iter_cman_only.index), 'observations from', len(df_iter_cds.index), ')')
        print(iter_prefix, ': CMAN only covered :', 'Proportion of dates presence :', 
              '{:.2%}'.format(len(ser_iter_cman_only.index.get_level_values(0).unique()) / len(df_iter_cds.index.get_level_values(0).unique())),
              '(', len(ser_iter_cman_only.index.get_level_values(0).unique()), 'dates from', len(df_iter_cds.index.get_level_values(0).unique()), ')')
        print(iter_prefix, ': CMAN only covered :', 'Proportion of countries presence :', 
              '{:.2%}'.format(len(ser_iter_cman_only.index.get_level_values(1).unique()) / len(df_iter_cds.index.get_level_values(1).unique())),
              '(', len(ser_iter_cman_only.index.get_level_values(1).unique()), 'countries from', len(df_iter_cds.index.get_level_values(1).unique()), ')')         
        print(iter_prefix, ': Both CMAN covered and CBIN covered :', 
              '{:.2%}'.format(len(df_iter_cds.loc[df_iter_cds['CMAN'].notna() & df_iter_cds['CBIN'].notna()].index) / len(df_iter_cds.index)),
              '(', len(df_iter_cds.loc[df_iter_cds['CMAN'].notna() & df_iter_cds['CBIN'].notna()].index), 'observations from', len(df_iter_cds.index), ')')
        df_iter_both = df_iter_cds.loc[df_iter_cds['CMAN'].notna() & df_iter_cds['CBIN'].notna()]
        print(iter_prefix, ': Both covered :', 'Proportion of dates presence :', 
              '{:.2%}'.format(len(df_iter_both.index.get_level_values(0).unique()) / len(df_iter_cds.index.get_level_values(0).unique())),
              '(', len(df_iter_both.index.get_level_values(0).unique()), 'dates from', len(df_iter_cds.index.get_level_values(0).unique()), ')')
        print(iter_prefix, ': Both covered :', 'Proportion of countries presence :', 
              '{:.2%}'.format(len(df_iter_both.index.get_level_values(1).unique()) / len(df_iter_cds.index.get_level_values(1).unique())),
              '(', len(df_iter_both.index.get_level_values(1).unique()), 'countries from', len(df_iter_cds.index.get_level_values(1).unique()), ')')   
        df_iter_both = df_iter_both.assign(Variation = (df_iter_both['CMAN'] - df_iter_both['CBIN']).abs().div(df_iter_both[['CMAN', 'CBIN']].mean(axis = 1)))
        print(iter_prefix, ': Both covered :', 'One observation countries number :', 
              '{:.2%}'.format(df_iter_both['CMAN'].groupby('Country').\
                              apply(lambda iter_group: 1 if iter_group.count() == 1 else 0).sum() / len(df_iter_both.index.get_level_values(1).unique())), 
              '(', df_iter_both['CMAN'].groupby('Country').apply(lambda iter_group: 1 if iter_group.count() == 1 else 0).sum(), 'countries from', 
              len(df_iter_both.index.get_level_values(1).unique()), ')')    
        print(iter_prefix, ': Both covered :', 'One observation dates number :', 
              '{:.2%}'.format(df_iter_both['CMAN'].groupby('Date').\
                              apply(lambda iter_group: 1 if iter_group.count() == 1 else 0).sum() / len(df_iter_both.index.get_level_values(0).unique())), 
              '(', df_iter_both['CMAN'].groupby('Date').apply(lambda iter_group: 1 if iter_group.count() == 1 else 0).sum(), 'dates from', 
              len(df_iter_both.index.get_level_values(0).unique()), ')')                
        print(iter_prefix, ': Both covered :', 'Equal values number :', 
              '{:.2%}'.format(len(df_iter_both[df_iter_both['Variation'] == 0].index) / len(df_iter_both.index)),
              '(', len(df_iter_both[df_iter_both['Variation'] == 0].index), 'observations from', len(df_iter_both.index), ')')
        print(iter_prefix, ': Both covered :', '0% < Variation coefficient <= 1%:', 
              '{:.2%}'.format(len(df_iter_both[(df_iter_both['Variation'] > 0) & (df_iter_both['Variation'] <= 0.01)].index) / len(df_iter_both.index)),
              '(', len(df_iter_both[(df_iter_both['Variation'] > 0) & (df_iter_both['Variation'] <= 0.01)].index), 'observations from', len(df_iter_both.index), ')')
        print(iter_prefix, ': Both covered :', '1% < Variation coefficient <= 5%:', 
              '{:.2%}'.format(len(df_iter_both[(df_iter_both['Variation'] > 0.01) & (df_iter_both['Variation'] <= 0.05)].index) / len(df_iter_both.index)),
              '(', len(df_iter_both[(df_iter_both['Variation'] > 0.01) & (df_iter_both['Variation'] <= 0.05)].index), 'observations from', len(df_iter_both.index), ')')
        print(iter_prefix, ': Both covered :', '5% < Variation coefficient <= 10%:', 
              '{:.2%}'.format(len(df_iter_both[(df_iter_both['Variation'] > 0.05) & (df_iter_both['Variation'] <= 0.10)].index) / len(df_iter_both.index)),
              '(', len(df_iter_both[(df_iter_both['Variation'] > 0.05) & (df_iter_both['Variation'] <= 0.10)].index), 'observations from', len(df_iter_both.index), ')')
        print(iter_prefix, ': Both covered :', '10% < Variation coefficient:', 
              '{:.2%}'.format(len(df_iter_both[df_iter_both['Variation'] > 0.10].index) / len(df_iter_both.index)),
              '(', len(df_iter_both[df_iter_both['Variation'] > 0.10].index), 'observations from', len(df_iter_both.index), ')')  
        df_iter_both_country_filled = df_iter_both.groupby('Country').apply(lambda iter_group: iter_group.droplevel([1, 2]).resample('B').ffill())
        ser_iter_country_corr = df_iter_both_country_filled.groupby('Country').apply(lambda iter_group: iter_group['CBIN'].corr(iter_group['CMAN'], min_periods = 5))     
        print(iter_prefix, ': Both covered :', 'Average by country correlation :', 
              '{:.2%}'.format(ser_iter_country_corr.mean()))
        plt_iter_country = ser_iter_country_corr.plot(figsize = (20, 5), title = iter_prefix + ': By country CMAN / CBIN correlation')
        plt_iter_country.set_xticks(range(len(ser_iter_country_corr.index)))
        plt_iter_country.set_xticklabels(ser_iter_country_corr.index.values)
        plt.show()           
        ser_iter_date_corr = df_iter_both.groupby('Date').apply(lambda iter_group: iter_group['CBIN'].corr(iter_group['CMAN'], min_periods = 5))        
        print(iter_prefix, ': Both covered :', 'Average by date correlation :', 
              '{:.2%}'.format(ser_iter_date_corr.mean()))
        plt_iter_date = ser_iter_date_corr.plot(figsize = (20, 5), title = iter_prefix + ': By date CMAN / CBIN correlation')
        plt.show()        

In [ ]:
df_cds_to_export = df_cds_mcap.reset_index('Market', drop = True)[['CMAN', 'CBIN']].sort_index(level = ['Country', 'Date']).\
                                        unstack('Country').swaplevel(axis = 1).sort_index(axis = 1)
df_cds_to_export.to_excel('Data_Files/Test_Files/CDS_Table.xlsx')